In [1]:
import csv
import numpy as np
import tensorflow as tf

from itertools import islice

In [4]:
def load_data(file_name = 'train.csv' , one_hot = True):
    csv_reader = csv.reader(open(file_name))

    train_set = []
    count = 0
    train_labels = []
    for row in islice(csv_reader ,1 , None):
        row = list(map(np.float32 , row)) #类型转换
        train_set.append(row)

        temp = [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
        temp[int(train_set[count][0])] = 1.
        train_labels.append(temp)

        count += 1
    
    for i in range(count):
        train_set[i] = train_set[i][1:785]
    
    train_set = np.matrix(train_set)
    train_labels = np.matrix(train_labels)
    
    return train_set , train_labels

In [5]:
#加载数据
train_set , train_labels = load_data()

In [6]:
train_set = train_set / 255.0
#print(train_set)
#print(train_labels)

In [17]:
mini_batch = 0

def next_batch(train_set , train_labels , batch_size = 128):
    global mini_batch
    if mini_batch + batch_size > 42000:
        mini_batch = 0

    train_set_batch = train_set[mini_batch : mini_batch + batch_size]
    train_labels_batch = train_labels[mini_batch : mini_batch + batch_size]
    
    mini_batch = mini_batch + batch_size
    
    return train_set_batch , train_labels_batch

In [32]:
lr = 0.001 #learning rate
training_iters = 100000 #epoch
batch_size = 128

n_inputs = 28 #图像为28列
n_steps = 25 #图像为28行
n_hidden_units = 128 #隐藏层神经元个数
n_classes = 10 #10分类问题

x = tf.placeholder(tf.float32 , [None , n_steps * n_inputs])
y = tf.placeholder(tf.float32 , [None , n_classes])

weights = {'in':tf.Variable(tf.random_normal([n_inputs , n_hidden_units])),
          'out':tf.Variable(tf.random_normal([n_hidden_units , n_classes]))}

biases = {'in':tf.Variable(tf.constant(0.1 , shape = [n_hidden_units,])),
         'out':tf.Variable(tf.constant(0.1 , shape = [n_classes,]))}

#=================
#定义函数

def RNN(X , weights , biases):
    #X 128batchsize 28 steps 28 inputs
    X = tf.reshape(X , [-1 , n_inputs])
    X_in = tf.matmul(X , weights['in']) + biases['in']
    X_in = tf.reshape(X_in , [-1 , n_steps , n_hidden_units])
    
    #=========
    #cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units , forget_bias=1.0 , state_is_tuple=True)
    _init_state = lstm_cell.zero_state(batch_size , dtype = tf.float32)
    
    outputs , states = tf.nn.dynamic_rnn(lstm_cell , X_in , initial_state=_init_state , time_major=False)
    
    result = tf.matmul(states[1] , weights['out'] + biases['out'])
    
    return result

In [33]:
prediction = RNN(x , weights , biases)

#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred , y))

#train_step = tf.train.AdamOptimizer(lr).minimize(cost)

#sess= tf.Session()
#sess.run(tf.global_variables_initializer())

#for i in range(10):
#    train_set_batch , train_labels_batch = next_batch(train_set , train_labels)
#    sess.run(train_step , feed_dict = {x:train_set_batch , y:train_labels_batch})
#    print(sess.run(loss , feed_dict = {x:train_set_batch , y:train_labels_batch}))

TypeError: raw_rnn() got an unexpected keyword argument 'initial_state'

In [24]:
sess.close()

NameError: name 'sess' is not defined

In [11]:
#dev-set
train_set_batch , train_labels_batch = next_batch(train_set , train_labels ,10000)

prediction_number = tf.argmax(prediction , 1)
real_number = tf.argmax(y , 1)
right = tf.equal(prediction_number , real_number)
accuracy = tf.reduce_mean(tf.cast(right , tf.float32))

print(sess.run(accuracy , feed_dict={x:train_set_batch , y:train_labels_batch} ))

#sess.close()

0.9233


In [89]:
def load_test_set(file_name = 'test.csv'):
    csv_reader = csv.reader(open(file_name))
    test_set = []
    
    for row in islice(csv_reader ,1 , None):
        row = list(map(np.float32 , row))
        test_set.append(row)
        
    test_set = np.matrix(test_set)

    return test_set

In [131]:
test_set = load_test_set()
test_set = test_set / 255.0

In [206]:
test_set_labels = tf.argmax(prediction , 1)
result = sess.run(test_set_labels , feed_dict={x:test_set})

In [207]:
csv_file = open('sample_submission.csv' , 'w' , newline='')
writer = csv.writer(csv_file)
writer.writerow(['ImageId','Label'])

for i in range(28000):
    writer.writerow([i+1 , result[i]])

csv_file.close()

In [3]:
28*28

784